In [1]:
pip install torch transformers datasets peft accelerate scikit-learn

ERROR: Could not find a version that satisfies the requirement torch (from versions: none)
ERROR: No matching distribution found for torch
Note: you may need to restart the kernel to use updated packages.


In [72]:
from transformers import AutoTokenizer
import pandas as pd
from sklearn.model_selection import train_test_split
import torch

# Defining LLM
local_llm = '/users/prashantnerkar/.ollama/models/meta-llama-3.2-1b'
#local_llm = '/users/prashantnerkar/.llama/checkpoints/Llama3.2-1B-Instruct'

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(local_llm, use_fast=False)
# model = TFAutoModel.from_pretrained(pretrained_weights)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
#    model.resize_token_embeddings(len(tokenizer))
    

# Example dataset
data = pd.DataFrame({
    "text": [
        "Unauthorized transaction on your credit card",
        "Routine check-up note with no abnormalities",
        "Large withdrawal from an unusual location",
        "Patient shows irregular drug prescription history",
        "Normal transaction at a grocery store",
    ],
    "label": [1, 0, 1, 1, 0]  # 1 for Fraud, 0 for Non-Fraud
})

# Train-test split
train_texts, test_texts, train_labels, test_labels = train_test_split(
    data['text'], data['label'], test_size=0.2
)

# Tokenize text
def tokenize_texts(texts, labels):
    tokens = tokenizer(
        texts.tolist(),
        padding=True,
        truncation=True,
        max_length=256,
        return_tensors="pt"
    )
    tokens["labels"] = torch.tensor(labels.tolist())
    return tokens

train_data = tokenize_texts(train_texts, train_labels)
test_data = tokenize_texts(test_texts, test_labels)

OSError: Incorrect path_or_model_id: '/users/prashantnerkar/.ollama/models/meta-llama-3.2-1b'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [61]:
from transformers import AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
from transformers import TrainingArguments, Trainer

# Load pre-trained LLaMA model
model = AutoModelForCausalLM.from_pretrained(local_llm, device_map="auto")

# LoRA configuration
peft_config = LoraConfig(
    r=8, 
    lora_alpha=16, 
    target_modules=["q_proj", "v_proj"], 
    lora_dropout=0.05, 
    bias="none", 
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, peft_config)

# Training arguments
training_args = TrainingArguments(
    output_dir="./llama-3.2-1b-fraud",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    learning_rate=5e-5,
    report_to="none"
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=tokenizer
)

# Train the model
trainer.train()


OSError: Error no file named pytorch_model.bin, model.safetensors, tf_model.h5, model.ckpt.index or flax_model.msgpack found in directory /users/prashantnerkar/.llama/checkpoints/Llama3.2-1B-Instruct.

In [43]:
from sklearn.metrics import classification_report

# Predict on test data
test_texts_tokens = tokenizer(
    test_texts.tolist(),
    padding=True,
    truncation=True,
    max_length=256,
    return_tensors="pt"
).to("cuda")

outputs = peft_model.generate(**test_texts_tokens)
predictions = [int(output.argmax(dim=-1)) for output in outputs]

# Classification report
print(classification_report(test_labels, predictions))


AssertionError: Torch not compiled with CUDA enabled

In [8]:
print(torch.cuda.is_available())

False
